In [21]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

# Load Dataset

In [13]:
path = Path.cwd().parents[0] / "input/data.csv"
df = pd.read_csv(path, encoding= "unicode_escape")

# Clean Dataset

In [79]:
def clean_df(df: pd.DataFrame):
    """Clean dataframe.
    """
    return (df
        .rename(columns=lambda c: (re.sub(r"([A-Z])", r" \1", c)
                                .lower()
                                .replace(" i d", " id")
                                .strip()
                                .replace(" ", "_")
                                )
        )
        .assign(
            invoice_no= lambda df_: df_["invoice_no"].astype(str).str.strip(), 
            stock_code= lambda df_: df_["stock_code"].astype(str).str.strip(), 
            description= lambda df_: df_["description"].str.title().str.strip(), 
            invoice_date=lambda df_: pd.to_datetime(df_["invoice_date"]),
            unit_price= lambda df_: df_["unit_price"].astype(float),
            country= lambda df_: df_["country"].str.title().str.strip(), 
            total_price=lambda df_: df_["quantity"].mul(df_["unit_price"]).astype(float),
        )
        .loc[:, ["invoice_no", "invoice_date", "description", "stock_code", "unit_price", 
                "quantity", "total_price", "customer_id", "country"]]
    )
df_clean = clean_df(df)

# EDA

In [80]:
(df_clean
    .describe()
    .transpose()
)

,count,mean,min,25%,50%,75%,max,std
invoice_date,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
unit_price,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
total_price,541909.0,17.987795,-168469.6,3.4,9.75,17.4,168469.6,378.810824
customer_id,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


In [87]:
(df_clean
    .loc[:, ["invoice_no", "stock_code", "customer_id", "country"]]
    .nunique().to_frame("num_unique_value")
)

,num_unique_value
invoice_no,25900
stock_code,4070
customer_id,4372
country,38


In [88]:
(df_clean
    .isna().sum()
    .to_frame("num_null_rows")
)

,num_null_rows
invoice_no,0
invoice_date,0
description,1454
stock_code,0
unit_price,0
quantity,0
total_price,0
customer_id,135080
country,0


Observations:
- The dataset comprises of sales from **1 Dec 2010 to 9 Dec 2011**. 
- There are more than 500k entries, consisting of 
    - 25k invoices
    - 4k types of item sold
    - 4k customers
    - 81k items sold
    - 160k revenue
- Customer ID and description has NULL rows.